# REF

- https://theaisummer.com/unet-architectures/
- [CTRANSCNN](https://www.sciencedirect.com/science/article/pii/S0950705123007803)
- [CrossViT]()

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
# from torchvision import models
from torchsummary import summary
import PIL
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import pandas as pd
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from pathlib import Path
from collections import defaultdict
from tqdm.notebook import tqdm
import validators

import argparse
import re
from collections import Counter

In [115]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [62]:
# X = torch.randn(1, 3, 512, 512)
# # X = nn.max_pool2d(X, kernel_size=2, stride=2)


# # torch.tensor_split(X, (16,16), dim=(2,3))
# X = torch.tensor_split(X.flatten(-2), 16*16)
# print(X[0].shape)
# print(X.shape)

# S = 128 # channel dim
# W = 227 # width
# H = 227 # height
# batch_size = 10

# image_new = X

# x = image_new.unsqueeze(0)

# size = 32 # patch size
# stride = 32 # patch stride
# patches = x.unfold(1, size, stride).unfold(2, size, stride).unfold(3, size, stride)
# print(patches.shape)

def split2SubImages(imgs: torch.tensor, subimage_size: tuple):
    imgs_shape = imgs.shape
    assert(len(imgs_shape) == 4) # B * C * Hi * Wi 
    assert(len(subimage_size) == 2) # Ho * Wo
    # check fully divisible
    assert( (imgs_shape[2] % subimage_size[0] == 0) and (imgs_shape[3] % subimage_size[1] == 0))
    
    # print(imgs)
    # print(subimage_size)
    # a1 = torch.nn.Unfold(kernel_size=subimage_size, dilation=subimage_size, stride=(1,1))(imgs)
    # a1 = torch.nn.Unfold(kernel_size=subimage_size)(imgs)
    a1 = torch.nn.Unfold(subimage_size, dilation=(1,1), stride=subimage_size)(imgs)
    # print(a1.shape)
    # print(imgs_shape)
    # print(a1)
    B = imgs.shape[0]
    C = imgs.shape[1]
    L = a1.shape[-1]
    a2 = a1.permute(0,2,1) # B * L * (C x Ho x Wo)
    a3 = a2.reshape((B,L,C)+subimage_size) 

    out_shape = a3.shape # B * num_imgs * C * Ho * Wo

    return a3 # B * num_imgs * C * Ho * Wo
    print(out_shape)
    # print(X.shape[:-1] + (2,2))
    # print(a1.shape)
    # #print(a1)
    # print(a2.shape)
    # print(a2)
    # print(a3.shape)
    # print(a3)
    # print(a4.shape)
    # print(a4)

imgs = torch.arange(6*6*3).reshape(1, 3, 6, 6).float()
# imgs = torch.randn(1, 3, 512, 512)
# print(imgs)
subimage_size = (2,2)
sub_imgs = split2SubImages(imgs, subimage_size)
# print(sub_imgs)
print(sub_imgs.shape)
keys = sub_imgs.reshape(sub_imgs.shape[:2] + (-1,)) # B * num_imgs * (CxHxW)
print(keys.shape )



torch.Size([1, 9, 3, 2, 2])
torch.Size([1, 9, 12])


In [116]:
imgs = torch.randn(1, 3, 512, 512)
subimage_size = (16,16)
img_t01 = split2SubImages(imgs, subimage_size)
k_t01 = img_t01.reshape(img_t01.shape[:2] + (-1,))
# print(sub_imgs.shape)
print(k_t01.shape)

print()
x2 = nn.MaxPool2d(kernel_size=2, stride=2)(imgs)
print(x2.shape)

img_t02 = split2SubImages(x2, subimage_size)
k_t02 = img_t02.reshape(img_t02.shape[:2] + (-1,))
print(k_t02.shape)

print()
x3 = nn.MaxPool2d(kernel_size=2, stride=2)(x2)
print(x3.shape)

img_t03 = split2SubImages(x3, subimage_size)
k_t03 = img_t03.reshape(img_t03.shape[:2] + (-1,))
print(k_t03.shape)


print()
x4 = nn.MaxPool2d(kernel_size=2, stride=2)(x3)
print(x4.shape)

img_t04 = split2SubImages(x4, subimage_size)
k_t04 = img_t04.reshape(img_t04.shape[:2] + (-1,))
print(k_t04.shape)


print()
x5 = nn.MaxPool2d(kernel_size=2, stride=2)(x4)
print(x5.shape)

img_t05 = split2SubImages(x5, subimage_size)
k_t05 = img_t05.reshape(img_t05.shape[:2] + (-1,))
print(k_t05.shape)

print()

torch.Size([1, 1024, 768])

torch.Size([1, 3, 256, 256])
torch.Size([1, 256, 768])

torch.Size([1, 3, 128, 128])
torch.Size([1, 64, 768])

torch.Size([1, 3, 64, 64])
torch.Size([1, 16, 768])

torch.Size([1, 3, 32, 32])
torch.Size([1, 4, 768])



In [77]:
imgs = torch.randn(1, 16, 64, 64)
subimage_size = (16,16)
img_t01 = split2SubImages(imgs, subimage_size) # B * num_imgs * C * Ho * Wo
B, num_imgs, C, Ho, Wo = img_t01.shape
k_t01 = img_t01.reshape((B, num_imgs*C, Ho*Wo))
# print(sub_imgs.shape)
print(k_t01.shape)

torch.Size([1, 256, 256])


In [111]:
class DownscaleCNN(nn.Module):
    def __init__(self, in_ch=3, out_ch=16):
        super(DownscaleCNN, self).__init__()

        
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        
        self.conv3 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        
        self.conv4 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(16)

        self.relu = nn.ReLU()
        self.max_pooling = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x1 = self.conv1(x)
        x1 = self.bn1(x1)
        x1 = self.relu(x1)

        x2 = self.max_pooling(x1)
        x2 = self.conv2(x2)
        x2 = self.bn2(x2)
        x2 = self.relu(x2)

        x3 = self.max_pooling(x2)
        x3 = self.conv3(x3)
        x3 = self.bn3(x3)
        x3 = self.relu(x3)
    
        x4 = self.max_pooling(x3)
        x4 = self.conv4(x4)
        x4 = self.bn4(x4)
        x4 = self.relu(x4)
    
        return x4, (x1, x2, x3, x4)


In [119]:
downscale_cnn = DownscaleCNN(in_ch=3, out_ch=16)
imgs = torch.randn(1, 3, 512, 512)
_, (x1, x2, x3, x4) = downscale_cnn(imgs)
print(imgs.shape)
print(x1.shape)
print(x2.shape)
print(x3.shape)
print(x4.shape)

# summary(downscale_cnn.to(device), (3,512,512), device=device.type)



subimage_size = (16,16)

for x in [x1, x2, x3, x4]:
    print()
    img_t01 = split2SubImages(x, subimage_size) # B * num_imgs * C * Ho * Wo
    B, num_imgs, C, Ho, Wo = img_t01.shape
    k_t01 = img_t01.reshape((B, num_imgs*C, Ho*Wo))
    # print(sub_imgs.shape)
    print(k_t01.shape)


torch.Size([1, 3, 512, 512])
torch.Size([1, 16, 512, 512])
torch.Size([1, 16, 256, 256])
torch.Size([1, 16, 128, 128])
torch.Size([1, 16, 64, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 512, 512]             448
       BatchNorm2d-2         [-1, 16, 512, 512]              32
              ReLU-3         [-1, 16, 512, 512]               0
         MaxPool2d-4         [-1, 16, 256, 256]               0
            Conv2d-5         [-1, 16, 256, 256]           2,320
       BatchNorm2d-6         [-1, 16, 256, 256]              32
              ReLU-7         [-1, 16, 256, 256]               0
         MaxPool2d-8         [-1, 16, 128, 128]               0
            Conv2d-9         [-1, 16, 128, 128]           2,320
      BatchNorm2d-10         [-1, 16, 128, 128]              32
             ReLU-11         [-1, 16, 128, 128]               0
        MaxPool2d-12

Using cuda for inference


In [9]:
# multihead_attn = nn.MultiheadAttention(128, 8, batch_first=True)
# query = torch.randn(10, 32, 128)
# key = query
# value = torch.randn(10, 32, 128)
# attn_output, attn_output_weights = multihead_attn(query, key, value, average_attn_weights=False)

# print(attn_output.shape)
# print(attn_output_weights.shape)

In [10]:
class MyViTBlock(nn.Module):
    def __init__(self, hidden_d, n_heads, mlp_ratio=4):
        super(MyViTBlock, self).__init__()
        self.hidden_d = hidden_d
        self.n_heads = n_heads

        # self.norm1 = nn.LayerNorm(hidden_d)
        # self.mhsa = MyMSA(hidden_d, n_heads)
        
        self.norm1_q = nn.LayerNorm(hidden_d)
        self.norm1_k = nn.LayerNorm(hidden_d)
        self.norm1_v = nn.LayerNorm(hidden_d)
        self.multihead_attn = nn.MultiheadAttention(hidden_d, n_heads, batch_first=True)
        
        self.norm2 = nn.LayerNorm(hidden_d)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_d, mlp_ratio * hidden_d),
            nn.GELU(),
            nn.Linear(mlp_ratio * hidden_d, hidden_d)
        )

    def forward(self, q, k, v):
        # x = v
        # out = x + self.mhsa(self.norm1(x))
        q1, k1, v1 = self.norm1_q(q), self.norm1_k(k), self.norm1_v(v)
        attn_output, attn_output_weights = self.multihead_attn(q1, k1, v1, average_attn_weights=False)
        out = v + attn_output
        
        out = out + self.mlp(self.norm2(out))
        return out

In [11]:
class FeatureTransformer(nn.Module):
    def __init__(self, data_dim=(256,512), hidden_d=512, n_heads=16, out_d=5) -> None:
        super(FeatureTransformer, self).__init__()
        
        assert(len(data_dim) == 2)

        self.data_dim = data_dim # (Length, embedding_dim)
        self.seq_len, self.emb_dim = data_dim
        self.hidden_d = hidden_d
        self.n_heads = n_heads
        self.out_d = out_d

        # 1) Input transformation
        self.linear_mapper = nn.Linear(data_dim[1], hidden_d)

        # 2) Positional embedding
        self.positional_embeddings = nn.Parameter(torch.randn(1, self.seq_len, hidden_d))
        # self.pos_embeddings = self.get_positional_embeddings(self.seq_len, hidden_d)


        # 3) Transformer encoder blocks
        # self.blocks = nn.ModuleList([MyViTBlock(hidden_d, n_heads) for _ in range(n_blocks)])
        self.block0 = MyViTBlock(hidden_d, n_heads)
        self.block1 = MyViTBlock(hidden_d, n_heads)
        self.block2 = MyViTBlock(hidden_d, n_heads)
        self.block3 = MyViTBlock(hidden_d, n_heads)

        # 4) Classification MLP
        self.mlp = nn.Sequential(
            nn.Linear(hidden_d, hidden_d),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(self.hidden_d, out_d),
            # nn.Softmax(dim=-1)
        )


    def forward(self, x5, v1, v2):
        B_size = x5.shape[0]

        tokens = self.linear_mapper(x5)
        
        # Adding classification token to the tokens
        # tokens = torch.stack([torch.vstack((self.class_token, tokens[i])) for i in range(len(tokens))])
        
        # Adding positional embedding
        pos_embed = self.positional_embeddings.repeat(B_size, 1, 1)
        out = tokens + pos_embed  
        
        # Transformer Blocks
        # for block in self.blocks:
        #     out2 = block(out, out, out)
            
        #     out = out2
        x_stage1 = self.block0(out, v1, v1) # Q, K, V
        x_stage2 = self.block1(x_stage1, x_stage1, x_stage1)
        x_stage3 = self.block2(x_stage2, v2, v2)
        x_stage4 = self.block3(x_stage3, x_stage3, x_stage3)

        out = x_stage4
            
        # Getting the classification token only
        out = out[:, 0]
        
        return self.mlp(out), out # Map to output dimension, output category distribution
            
    
    def get_positional_embeddings(self, sequence_length, d):
        result = torch.ones(sequence_length, d)
        for i in range(sequence_length):
            for j in range(d):
                result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
        return result
        

In [12]:
# class MyViT(nn.Module):
#     def __init__(self, chw=(1, 28, 28), n_patches=7, n_blocks=2, hidden_d=128, n_heads=2, out_d=10):
#         # Super constructor
#         super(MyViT, self).__init__()

#         # Attributes
#         self.chw = chw # (C, H, W)
#         self.n_patches = n_patches
#         self.n_blocks = n_blocks
#         self.n_heads = n_heads
#         self.hidden_d = hidden_d

#         assert chw[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
#         assert chw[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"
#         self.patch_size = (chw[1] / n_patches, chw[2] / n_patches)

#         # 1) Linear mapper
#         self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
#         self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)

#         # 2) Learnable classifiation token
#         self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))
        
#         # # 3) Positional embedding
#         # self.positional_embeddings = nn.Parameter(torch.tensor(self.get_positional_embeddings(self.n_patches ** 2 + 1, self.hidden_d)))
#         # self.positional_embeddings.requires_grad = False
        
        
#         # 3) Positional embedding
#         self.register_buffer('positional_embeddings', self.get_positional_embeddings(n_patches ** 2 + 1, hidden_d), persistent=False)
        
#         # 4) Transformer encoder blocks
#         self.blocks = nn.ModuleList([MyViTBlock(hidden_d, n_heads) for _ in range(n_blocks)])
           
#         # 5) Classification MLPk
#         self.mlp = nn.Sequential(
#             nn.Linear(self.hidden_d, out_d),
#             nn.Softmax(dim=-1)
#         )
        

#     def forward(self, images):
#         n, c, h, w = images.shape
#         patches = self.patchify(images, self.n_patches)
#         # print(patches)
#         tokens = self.linear_mapper(patches)
        
#         # Adding classification token to the tokens
#         tokens = torch.stack([torch.vstack((self.class_token, tokens[i])) for i in range(len(tokens))])
        
#         # Adding positional embedding
#         pos_embed = self.positional_embeddings.repeat(n, 1, 1)
#         out = tokens + pos_embed  
        
#         # Transformer Blocks
#         for block in self.blocks:
#             out2 = block(out, out, out)
            
#             out = out2
            
#         # Getting the classification token only
#         out = out[:, 0]
        
#         return self.mlp(out), out # Map to output dimension, output category distribution
            

#     def patchify(self, images, n_patches):
#         assert len(images.shape) == 4
#         b,c,h,w = images.shape

#         size_h = int(h/n_patches) # patch size
#         stride_h = size_h # patch stride
#         size_w = int(w/n_patches) # patch size
#         stride_w = size_w # patch stride
#         x = images.permute((0,2,3,1))
#         x2 = x.unfold(1, size_h, stride_h).unfold(2, size_w, stride_w).flatten(start_dim=1, end_dim=2)
#         # print(x2.shape, "x2 pachify") # x2.shape = (b,p**2, c,h,w)
#         x3 = x2.permute((0,1,3,4,2)).flatten(start_dim=2, end_dim=4)
#         # print(x3.shape, "x3 pachify") # x3.shape = (b,p**2,h*w*c)
        
#         return x3

#     def get_positional_embeddings(self, sequence_length, d):
#         result = torch.ones(sequence_length, d)
#         for i in range(sequence_length):
#             for j in range(d):
#                 result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
#         return result



In [13]:
import sys
import os
sys.path.append(os.getcwd() + '/../')
from lib.network_architecture.unet_transformer_01 import MyViTBlock, FeatureTransformer, Unet, BridgingModel, \
                                                         get_unet_transformer_model_output
import torch

In [14]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [15]:
unet_model = Unet(3, 3).to(device)
bridging_model = BridgingModel().to(device)
feature_transformer_model = FeatureTransformer(data_dim=(256,512), hidden_d=512, n_heads=16, out_d=5).to(device)


In [16]:
X = torch.randn(1, 3, 256, 256).to(device)
# out, hidden_features = unet_model(X)
# # (x1, x2, x3, x4, x5) = hidden_features
# fs = bridging_model(*hidden_features)
# B, C, H, W = fs[0].shape
# vs = [x.permute(0,2,3,1).reshape(B, H*W, C) for x in fs]


# for v in vs:
#     print(v.shape)


# x5 = hidden_features[4].permute(0,2,3,1).reshape(B, H*W, C)
# print(x5.shape)

# y_pred, out = feature_transformer_model(x5, vs[0], vs[1])

y_pred, out, unet_out = get_unet_transformer_model_output(X, unet_model, bridging_model, feature_transformer_model)

print(y_pred, torch.argmax(y_pred, dim=-1))
print(unet_out.shape)



tensor([[-0.1291,  0.1260, -0.1129, -0.1690, -0.0675]], device='cuda:0',
       grad_fn=<AddmmBackward0>) tensor([1], device='cuda:0')
torch.Size([1, 3, 256, 256])


In [17]:
from torchsummary import summary
X = torch.randn(2, 3, 256*2, 256*2).to(device)
out, hidden_features = unet_model(X)
# (x1, x2, x3, x4, x5) = hidden_features
fs = bridging_model(*hidden_features)
B, C, H, W = fs[0].shape
vs = [x.permute(0,2,3,1).reshape(B, H*W, C) for x in fs]

x5 = hidden_features[4].permute(0,2,3,1).reshape(B, H*W, C)
# print(x5.shape)

feature_transformer_model = FeatureTransformer(data_dim=x5.shape[1:], hidden_d=512, n_heads=16, out_d=5).to(device)

y_pred, out = feature_transformer_model(x5, vs[0], vs[1])

summary(unet_model, (3, 512, 512), device=device.type)
summary(bridging_model, [ x.shape[1:] for x in hidden_features ], device=device.type)
# print(x5.shape[1:], vs[0].shape[1:], vs[1].shape[1:])
# summary(feature_transformer_model, [ x5.shape[1:], vs[0].shape[1:], vs[1].shape[1:] ], device=device.type)

# bridging_model = BridgingModel().to(device)
# feature_transformer_model = FeatureTransformer().to(device)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           1,792
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
            Conv2d-4         [-1, 64, 512, 512]          36,928
       BatchNorm2d-5         [-1, 64, 512, 512]             128
              ReLU-6         [-1, 64, 512, 512]               0
        DoubleConv-7         [-1, 64, 512, 512]               0
            InConv-8         [-1, 64, 512, 512]               0
         MaxPool2d-9         [-1, 64, 256, 256]               0
           Conv2d-10        [-1, 128, 256, 256]          73,856
      BatchNorm2d-11        [-1, 128, 256, 256]             256
             ReLU-12        [-1, 128, 256, 256]               0
           Conv2d-13        [-1, 128, 256, 256]         147,584
      BatchNorm2d-14        [-1, 128, 2